In [1]:
import os
import importlib
import time

import numpy as np
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import skimage
from skimage import transform as skimg_transform
#from skimage import io as img_io
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, utils, models
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler

import load_data
import model
import warnings

In [2]:
plt.style.use('Solarize_Light2')
warnings.filterwarnings('ignore')
%matplotlib inline
importlib.reload(load_data)
importlib.reload(model)

<module 'model' from '/home/uldo/work/kaggle/competitions/Humpback_Whale_Identification/code/model.py'>

In [3]:
#BATCH_SIZE = 100
#BATCH_SIZE = 32
#BATCH_SIZE = 200
#BATCH_SIZE = 128
#BATCH_SIZE = 64
BATCH_SIZE = 32
#IMAGE_W = 100
IMAGE_W = 256
#IMAGE_H = 100
IMAGE_H = 256

In [4]:
'''
class HWI_ConvNeuralNet(nn.Module):
    
    def __init__(self):
        super(HWI_ConvNeuralNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 6, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 12, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.drop_out = nn.Dropout()
        self.out1 = nn.Linear(int(12 * IMAGE_W/4 * IMAGE_H/4), 900)
        self.out2 = nn.Linear(900, 1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.drop_out(x)
        output = self.out1(x)
        output = self.out2(output)
        #return output, x
        return output[:, 0]
        #return F.log_softmax(output, dim=1)
'''

'\nclass HWI_ConvNeuralNet(nn.Module):\n    \n    def __init__(self):\n        super(HWI_ConvNeuralNet, self).__init__()\n        self.conv1 = nn.Sequential(\n            nn.Conv2d(3, 6, 5, 1, 2),\n            nn.ReLU(),\n            nn.MaxPool2d(kernel_size=2)\n        )\n        self.conv2 = nn.Sequential(\n            nn.Conv2d(6, 12, 5, 1, 2),\n            nn.ReLU(),\n            nn.MaxPool2d(kernel_size=2)\n        )\n        self.drop_out = nn.Dropout()\n        self.out1 = nn.Linear(int(12 * IMAGE_W/4 * IMAGE_H/4), 900)\n        self.out2 = nn.Linear(900, 1)\n        \n    def forward(self, x):\n        x = self.conv1(x)\n        x = self.conv2(x)\n        x = x.view(x.size(0), -1)\n        output = self.drop_out(x)\n        output = self.out1(x)\n        output = self.out2(output)\n        #return output, x\n        return output[:, 0]\n        #return F.log_softmax(output, dim=1)\n'

In [5]:
class HWI_ConvNeuralNet(nn.Module):
    
    def __init__(self):
        super(HWI_ConvNeuralNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 7, padding=1)
        self.conv2_bn = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)        
        self.pool2 = nn.AvgPool2d(3, 3)
        
        self.fc1 = nn.Linear(64 * 4 * 4 * 16, 1024)
        #self.fc1 = nn.Linear(64, 1024)
        #self.fc2 = nn.Linear(1024, 5004)
        self.fc2 = nn.Linear(1024, 1)
        
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        #print("forward, input x.size(): ", x.size())
        x = self.pool(F.relu(self.conv2_bn(self.conv1(x))))
        #print("forward, x.size() after self.pool(F.relu(self.conv2_bn(self.conv1(x)))): ", x.size())
        x = self.pool2(F.relu(self.conv2(x)))
        #print("forward, x.size() after self.pool2(F.relu(self.conv2(x))): ", x.size())
        x = x.view(-1, 64 * 4 * 4 * 16)
        #print("forward, x.size() after x.view(-1, 64 * 4 * 4 * 16): ", x.size())
        x = F.relu(self.fc1(x))
        #print("forward, x.size() after F.relu(self.fc1(x)): ", x.size())
        x = self.dropout(x)
        #print("forward, x.size() after self.dropout(x): ", x.size())
        x = self.fc2(x)
        #print("forward, x.size() after self.fc2(x): ", x.size())
        #x = self.fc3(x)
        x = self.sigmoid(x)
        #print("forward, x.size() after self.sigmoid(x): ", x.size())
        return x       

In [6]:
class HWI_ConvNeuralNet(nn.Module):
    
    def __init__(self):
        super(HWI_ConvNeuralNet, self).__init__()
        '''
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 6, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 12, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.drop_out = nn.Dropout()
        self.out1 = nn.Linear(int(12 * IMAGE_W/4 * IMAGE_H/4), 900)
        self.out2 = nn.Linear(900, 1)
        '''
        self.conv1 = nn.Conv2d(3, 32, kernel_size=9, stride=2, padding=4)
        #self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2)
        #self.fc1 = nn.Linear(int(IMAGE_H * IMAGE_W 131072 x 16), 1024)
        #self.fc1 = nn.Linear(524288 * 64, 1024)
        self.fc1 = nn.Linear(32768 * 16, 1024)
        self.fc2 = nn.Linear(1024, 32)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.5)
        
        #forward, input x.size():  torch.Size([64, 3, 256, 256])
        #forward, after F.relu(self.conv1(x)) x.size():  torch.Size([64, 32, 128, 128])
        #forward, after self.pool(x) x.size():  torch.Size([64, 32, 64, 64])
        #forward, after F.relu(self.conv2(x)) x.size():  torch.Size([64, 64, 32, 32])
        #forward, after self.pool(x) x.size():  torch.Size([64, 64, 16, 16])
        #forward, after x.view(-1, 65536 * 16) x.size():  torch.Size([1, 1048576])
        #forward, after F.relu(self.fc1(x)) x.size():  torch.Size([1, 1024])
        #forward, after self.dropout x.size():  torch.Size([1, 1024])
        #forward, after self.fc2 x.size():  torch.Size([1, 1])
        #forward, after self.sigmoid(x) x.size():  torch.Size([1, 1])
        
    def forward(self, x):
        '''
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.drop_out(x)
        output = self.out1(x)
        output = self.out2(output)
        #return output, x
        return output[:, 0]
        '''
        #print("forward, input x.size(): ", x.size())
        
        x = F.relu(self.conv1(x))
        #print("forward, after F.relu(self.conv1(x)) x.size(): ", x.size())
        x = self.pool(x)
        #print("forward, after self.pool(x) x.size(): ", x.size())
        x = F.relu(self.conv2(x))
        #print("forward, after F.relu(self.conv2(x)) x.size(): ", x.size())
        x = self.pool(x)     
        #print("forward, after self.pool(x) x.size(): ", x.size())  
        x = x.view(-1, 32768 * 16)
        #print("forward, after x.view(-1, 65536 * 16) x.size(): ", x.size())
        x = F.relu(self.fc1(x))
        #print("forward, after F.relu(self.fc1(x)) x.size(): ", x.size())
        x = self.dropout(x)
        #print("forward, after self.dropout x.size(): ", x.size())
        x = self.fc2(x)
        #print("forward, after self.fc2 x.size(): ", x.size())
        x = self.sigmoid(x)
        #print("forward, after self.sigmoid(x) x.size(): ", x.size())
        return x
        

In [7]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        orig_height, orig_width = image.shape[:2]
        if isinstance(self.output_size, int):
            if orig_height > orig_width:
                new_height, new_width = self.output_size * orig_height / orig_width, self.output_size
            else:
                new_height, new_width = self.output_size, self.output_size * orig_width / orig_height
        else:
            new_height, new_width = self.output_size

        new_height, new_width = int(new_height), int(new_width)

        img = skimg_transform.resize(image, (new_height, new_width))

        return {'image': img, 'label': label}

In [8]:
class UnifyRescale(object):
    
    def __init__(self, output_size=128):
        assert isinstance(output_size, int)
        self.output_size = output_size
        
    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        img = skimg_transform.resize(image, (self.output_size, self.output_size))

        return {'image': img, 'label': label}

In [9]:
class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        orig_height, orig_width = image.shape[:2]
        new_height, new_width = self.output_size

        top = np.random.randint(0, orig_height - new_height)
        left = np.random.randint(0, orig_width - new_width)

        image = image[top: top + new_height, left: left + new_width]

        return {'image': image, 'label': label}

In [10]:
class ToTensor(object):
    
    """Convert ndarrays in sample to Tensors."""
    def __init__(self, image_size=128):
        self.image_size = image_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        
        """ The original code didn't expect gray scale images """
        
        gray_scale_image = torch.zeros(
            [self.image_size, self.image_size]
        ).shape == image.shape
        if gray_scale_image:
            image = np.stack((image,) * 3, axis=-1)
        
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image), 'label': torch.tensor(label, dtype=torch.uint8)}

In [11]:
def show_whale_batch(sample_batched):
    """Show whales for a batch of samples."""
    images_batch = sample_batched['image']
    labels_batch = sample_batched['label']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)

    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

    for i in range(batch_size):
        plt.title('Batch from dataloader')

In [12]:
def init_weights(submodule):
    if type(submodule) == nn.Linear:
        nn.init.xavier_uniform_(submodule.weight)
        submodule.bias.data.fill_(0.01)

In [13]:
def prepare_loaders(dataset, valid_train_ratio=0.6):
    dataset_size = len(dataset)
    print("dataset_size: ", dataset_size)

    validation_subset_size = int(dataset_size * (1 - valid_train_ratio))
    print("validation_subset_size: ", validation_subset_size)

    indices = list(range(dataset_size))
    validation_indices = np.random.choice(indices, size=validation_subset_size, replace=False)
    train_indices = list(set(indices) - set(validation_indices))

    train_sampler = SubsetRandomSampler(train_indices)
    validation_sampler = SubsetRandomSampler(validation_indices)
    
    dataset_sizes = {
            'train': len(train_indices),
            'validation': len(validation_indices)
        }

    #train_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=train_sampler, pin_memory=True)
    train_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=train_sampler)
    #validation_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=validation_sampler, pin_memory=True)
    validation_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=validation_sampler)
    loaders = {
            'train': train_loader,
            'validation': validation_loader
        }

    return loaders, dataset_sizes

In [14]:
def unfold_batch(batch):
    return batch['image'], batch['label']

In [15]:
def one_epoch_train(model, data_loader, criterion, optimizer):
    
    accuracy = 0.0
    total_loss = 0.0
    correct_predicted_total = 0.0
    
    for i, data_batch in enumerate(data_loader, 0):
        #print("one_epoch_model_process, len(data_batch): ", len(data_batch))
        #print("one_epoch_model_process, type(data_batch): ", type(data_batch))        
        
        inputs, labels = unfold_batch(data_batch)
        #print("one_epoch_model_process, inputs.size(): ", inputs.size())
        if inputs.size()[0] == BATCH_SIZE:
            
            #print("inputs: ", inputs)
            #print("labels: ", labels)
        
            inputs = inputs.to(device, dtype=torch.float)
            #labels = labels.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.long)
            optimizer.zero_grad()
        
            
            outputs = model(inputs)

            #print("outputs.size():\n", outputs.size())
            #print("labels.size():\n", labels.size())
            #print("outputs:\n", outputs)
            #print("outputs[:, 0]:\n", outputs[:, 0])
            #print("labels:\n", labels)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            total_loss += loss.item() * inputs.size(0)
        
            predicted = outputs > 0
            #print("type(predicted): ", type(predicted))
            #print("predicted:\n", predicted)

            #total += labels.size(0)
            labels = labels.data.byte()
            #print("predicted == labels:\n", predicted == labels)
            #sum_of_correct_predicted = torch.sum((predicted == labels).all(1))
            sum_of_correct_predicted = torch.sum((predicted == labels))
            item = sum_of_correct_predicted.item()
            correct_predicted_total += item
        
    accuracy = correct_predicted_total
    
    #epoch_train_loss = total_loss / train_dataset_size
    #epoch_train_accuracy = correct_predicted_total / train_dataset_size
    return (total_loss, accuracy)

In [16]:
def validate(model, validation_loader, criterion):
    model.eval()
    with torch.no_grad():
        
        correct_predicted_total = 0.0
        total_loss = 0.0
        #total = 0.0
        
        for data_batch in validation_loader:
            inputs, labels = unfold_batch(data_batch)

            inputs = inputs.to(device, dtype=torch.float)
            #labels_as_float = labels.to(device, dtype=torch.float)
            #labels = labels.to(device, dtype=torch.uint8)
            labels = labels.to(device, dtype=torch.float)
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            #loss = criterion(outputs, labels_as_float)
            total_loss += loss.item() * inputs.size(0)
            
            predicted = outputs > 0

            #total += labels.size(0)
            labels = labels.data.byte()
            #sum_of_correct_predicted = torch.sum((predicted == labels).all(1))
            sum_of_correct_predicted = torch.sum((predicted == labels))
            item = sum_of_correct_predicted.item()

            correct_predicted_total += item

        #accuracy = correct_predicted_total / total
        accuracy = correct_predicted_total
        

    return (total_loss, accuracy)

In [17]:
def one_epoch_validate(model, data_loader, criterion):
    model.eval()
    with torch.no_grad():
        
        correct_predicted_total = 0.0
        total_loss = 0.0

        for data_batch in data_loader:
            inputs, labels = unfold_batch(data_batch)
            if inputs.size()[0] == BATCH_SIZE:

                inputs = inputs.to(device, dtype=torch.float)
                #labels = labels.to(device, dtype=torch.float)
                labels = labels.to(device, dtype=torch.long)
                outputs = model(inputs)
            
                loss = criterion(outputs, labels)
                total_loss += loss.item() * inputs.size(0)
            
                predicted = outputs > 0
            
                labels = labels.data.byte()
                sum_of_correct_predicted = torch.sum((predicted == labels))
                item = sum_of_correct_predicted.item()

                correct_predicted_total += item

        accuracy = correct_predicted_total        

    return (total_loss, accuracy)

In [18]:
def one_epoch_model_process(model, loader, criterion, optimizer=None):
    accuracy = 0.0
    total_loss = 0.0

    for data_batch in loader:
        print("one_epoch_model_process, len(data_batch): ", len(data_batch))
        print("one_epoch_model_process, type(data_batch): ", type(data_batch))
        if len(data_batch) == BATCH_SIZE:
            inputs, labels = unfold_batch(data_batch)
            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)
            outputs = model(inputs)[:, 0]
            
            loss = criterion(outputs, labels)
            if optimizer:
                loss.backward()
                optimizer.step()
            total_loss += loss.item() * inputs.size(0)
            
            predicted = outputs > 0

            labels = labels.data.byte()
            #torch_sum_of_correct_predicted = torch.sum((predicted == labels).all(1))
            torch_sum_of_correct_predicted = torch.sum((predicted == labels))
            correct_predicted = torch_sum_of_correct_predicted.item()

    accuracy += correct_predicted        

    return (total_loss, accuracy)

In [19]:
def train_model(num_of_epoch, model, dataset_loaders, dataset_sizes, criterion, optimizer):
    torch.cuda.empty_cache()
    since = time.time()
    
    train_loader = dataset_loaders['train']
    validation_loader = dataset_loaders['validation']
    train_dataset_size = dataset_sizes['train']
    validation_dataset_size = dataset_sizes['validation']
    
    best_model_accuracy = 0.0
    best_model_weights = model.state_dict()
    
    train_losses = []
    train_accuracies = []
    validation_losses = []
    validation_accuracies = []
    
    for epoch in range(num_of_epoch):
        
        train_loss, train_accuracy = one_epoch_train(model, train_loader, criterion, optimizer)
        train_losses.append(train_loss / train_dataset_size)
        train_accuracies.append(train_accuracy / train_dataset_size)
        
        validation_loss, validation_accuracy = one_epoch_validate(model, validation_loader, criterion)
        validation_losses.append(validation_loss / validation_dataset_size)
        validation_accuracies.append(validation_accuracy / validation_dataset_size)
        
        if validation_accuracy > best_model_accuracy:
            best_model_accuracy = validation_accuracy
            best_model_weights = model.state_dict()
        
        print("Epoch {}: train loss {}, train accuracy"
          " {}, validation loss {}, validation accuracy {}".format(
              epoch + 1,
              train_loss / train_dataset_size,
              train_accuracy / train_dataset_size,
              validation_loss / validation_dataset_size,
              validation_accuracy / validation_dataset_size
            )
        )
    print("Finished Training")
    time_elapsed = time.time() - since
    print(
            'Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60
        )
    )
    print("Best model accuracy: ", best_model_accuracy)
    model.load_state_dict(best_model_weights)
    return train_losses, validation_losses, train_accuracies, validation_accuracies

In [20]:
def run_model(model, batch):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    #inputs = batch
    #inputs = inputs.to(device, dtype=torch.float)
    inputs = batch.to(device, dtype=torch.float)
    outputs = model(inputs)
    #return outputs[0].cpu()
    return outputs.cpu()

In [21]:
def save_model(model, full=True, name='model'):
    if not full:
        torch.save(model.state_dict(), '{}_params.pkl'.format(name))
    else:
        torch.save(model, '{}.pkl'.format(name))
    return

In [22]:
def restore_model(name='model'):
    return torch.load('{}.pkl'.format(name))

In [23]:
train_df = load_data.load_text_data('../input/train.csv')

In [24]:
train_df_two_classes = train_df.copy()

In [25]:
train_df_two_classes.loc[train_df_two_classes['Id'] != 'new_whale', 'Id'] = 'not_new_whale'

In [26]:
train_df_two_classes.head(40)

,Image,Id
0,0000e88ab.jpg,not_new_whale
1,0001f9222.jpg,not_new_whale
2,00029d126.jpg,not_new_whale
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale
5,0006e997e.jpg,new_whale
6,000a6daec.jpg,not_new_whale
7,000f0f2bf.jpg,new_whale
8,0016b897a.jpg,not_new_whale
9,001c1ac5f.jpg,not_new_whale


In [27]:
'''
image_size = 128
dataset = load_data.HumpbackWhalesDataset(
    train_df_two_classes,
    #train_df,
    #transform=load_data.transforms.ToTensor()
    #transform=ToTensor()
    transform=transforms.Compose(
        [
            #Rescale(int(image_size*1.25)),
            Rescale(int(image_size)),
            #RandomCrop(image_size),
            UnifyRescale(int(image_size)),
            ToTensor()
        ]
    )
)
'''

'\nimage_size = 128\ndataset = load_data.HumpbackWhalesDataset(\n    train_df_two_classes,\n    #train_df,\n    #transform=load_data.transforms.ToTensor()\n    #transform=ToTensor()\n    transform=transforms.Compose(\n        [\n            #Rescale(int(image_size*1.25)),\n            Rescale(int(image_size)),\n            #RandomCrop(image_size),\n            UnifyRescale(int(image_size)),\n            ToTensor()\n        ]\n    )\n)\n'

In [28]:
#image_size = 128
#image_size = 100
dataset = load_data.HumpbackWhalesDataset(
    train_df_two_classes,
    #train_df,
    #transform=load_data.transforms.ToTensor()
    #transform=ToTensor()
    transform=transforms.Compose(
        [
            transforms.Resize((IMAGE_W, IMAGE_H)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ]
    )
)

In [29]:
dataset_loaders, dataset_sizes = prepare_loaders(dataset)

dataset_size:  25361
validation_subset_size:  10144


In [30]:
train_loader = dataset_loaders['train']
validation_loader = dataset_loaders['validation']
train_dataset_size = dataset_sizes['train']
validation_dataset_size = dataset_sizes['validation']

In [31]:
print(train_dataset_size)
print(validation_dataset_size)

15217
10144


In [32]:
print(train_loader)

In [33]:
train_iter = iter(train_loader)

Process Process-1:
Traceback (most recent call last):
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/multiprocessing/conne

In [34]:
print(type(train_iter))

<class 'torch.utils.data.dataloader._DataLoaderIter'>


In [35]:
#images, labels = train_iter.next()
sample = train_iter.next()
print(sample)

{'image': tensor([[[[ 2.2489,  2.2489,  2.1975,  ...,  2.1975,  2.1975,  2.1804],
          [ 2.2489,  2.2147,  2.1633,  ...,  2.1290,  2.0948,  2.0605],
          [ 2.2318,  2.1804,  2.1290,  ...,  2.1975,  2.1633,  2.1462],
          ...,
          [ 2.1119,  2.0948,  2.0777,  ...,  2.0605,  2.0777,  2.1119],
          [ 1.5982,  1.7180,  1.8550,  ...,  2.1633,  2.1290,  2.0434],
          [ 1.6838,  1.6667,  1.7352,  ...,  2.1975,  2.2147,  2.1804]],

         [[ 2.4286,  2.4286,  2.3761,  ...,  2.3761,  2.3761,  2.3585],
          [ 2.4286,  2.3936,  2.3410,  ...,  2.3060,  2.2710,  2.2360],
          [ 2.4111,  2.3585,  2.3060,  ...,  2.3761,  2.3410,  2.3235],
          ...,
          [ 2.2885,  2.2710,  2.2535,  ...,  2.2360,  2.2535,  2.2885],
          [ 1.7633,  1.8859,  2.0259,  ...,  2.3410,  2.3060,  2.2185],
          [ 1.8508,  1.8333,  1.9034,  ...,  2.3761,  2.3936,  2.3585]],

         [[ 2.6400,  2.6400,  2.5877,  ...,  2.5877,  2.5877,  2.5703],
          [ 2.6400, 

In [36]:
'''
for i_batch, sample_batched in enumerate(train_loader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['label'])
    # observe 4th batch and stop.
    if i_batch == 0:
        plt.figure(figsize=(24, 24))
        show_whale_batch(sample_batched)
        plt.axis('off')
        #plt.ioff()
        plt.show()
        break
'''

"\nfor i_batch, sample_batched in enumerate(train_loader):\n    print(i_batch, sample_batched['image'].size(),\n          sample_batched['label'])\n    # observe 4th batch and stop.\n    if i_batch == 0:\n        plt.figure(figsize=(24, 24))\n        show_whale_batch(sample_batched)\n        plt.axis('off')\n        #plt.ioff()\n        plt.show()\n        break\n"

In [37]:
dataset.encoding

{'not_new_whale': 0, 'new_whale': 1}

In [38]:
#hwi_conv_neural_net = HWI_ConvNeuralNet()
#hwi_conv_neural_net.apply(init_weights)

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [40]:
print(device)

cuda:0


In [41]:
#hwi_conv_neural_net.to(device)

In [42]:
#criterion = nn.BCEWithLogitsLoss(reduction='sum')
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.BCELoss()
#optimizer = optim.Adam(hwi_conv_neural_net.parameters(), lr=0.001, betas=(0.9, 0.99))

In [43]:
num_of_epoch = 12
#train_losses = []
#train_accuracies = []
#validation_losses = []
#validation_accuracies = []

In [44]:
'''
torch.cuda.empty_cache()
since = time.time()
for epoch in range(num_of_epoch):
    epoch_train_accuracy = 0.0
    epoch_train_loss = 0.0
    correct_predicted_total = 0.0
    train_loss = 0.0
    #total_loss = 0.0
    #total = 0.0
    for i, data_batch in enumerate(train_loader, 0):
        
        inputs = data_batch['image']
        labels = data_batch['label']
        print("inputs: ", inputs)
        print("labels: ", labels)
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        optimizer.zero_grad()
        
        outputs = hwi_conv_neural_net(inputs)

        print("outputs.size():\n", outputs.size())
        print("labels.size():\n", labels.size())
        print("outputs:\n", outputs)
        #print("outputs[:, 0]:\n", outputs[:, 0])
        print("labels:\n", labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        
        predicted = outputs > 0
        print("type(predicted): ", type(predicted))
        print("predicted:\n", predicted)

        #total += labels.size(0)
        labels = labels.data.byte()
        print("predicted == labels:\n", predicted == labels)
        #sum_of_correct_predicted = torch.sum((predicted == labels).all(1))
        sum_of_correct_predicted = torch.sum((predicted == labels))
        item = sum_of_correct_predicted.item()
        correct_predicted_total += item
        
    epoch_train_loss = train_loss / train_dataset_size
    epoch_train_accuracy = correct_predicted_total / train_dataset_size
    
    validation_loss, validation_accuracy = validate(hwi_conv_neural_net, validation_loader, criterion)
    
    epoch_validation_loss = validation_loss / validation_dataset_size
    epoch_validation_accuracy = validation_accuracy / validation_dataset_size
    
    print("Epoch {}: train loss {}, train accuracy"
          " {}, validation loss {}, validation accuracy {}".format(
              epoch + 1,
              epoch_train_loss,
              epoch_train_accuracy,
              epoch_validation_loss,
              epoch_validation_accuracy
        )
    )
    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_accuracy)
    validation_losses.append(epoch_validation_loss)
    validation_accuracies.append(epoch_validation_accuracy)
            
print("Finished Training")
time_elapsed = time.time() - since
print(
    'Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60
    )
)
'''

'\ntorch.cuda.empty_cache()\nsince = time.time()\nfor epoch in range(num_of_epoch):\n    epoch_train_accuracy = 0.0\n    epoch_train_loss = 0.0\n    correct_predicted_total = 0.0\n    train_loss = 0.0\n    #total_loss = 0.0\n    #total = 0.0\n    for i, data_batch in enumerate(train_loader, 0):\n        \n        inputs = data_batch[\'image\']\n        labels = data_batch[\'label\']\n        print("inputs: ", inputs)\n        print("labels: ", labels)\n        inputs = inputs.to(device, dtype=torch.float)\n        labels = labels.to(device, dtype=torch.float)\n        optimizer.zero_grad()\n        \n        outputs = hwi_conv_neural_net(inputs)\n\n        print("outputs.size():\n", outputs.size())\n        print("labels.size():\n", labels.size())\n        print("outputs:\n", outputs)\n        #print("outputs[:, 0]:\n", outputs[:, 0])\n        print("labels:\n", labels)\n        loss = criterion(outputs, labels)\n        loss.backward()\n        optimizer.step()\n        \n        trai

In [45]:
resnet18 = models.resnet18(pretrained=True)

In [46]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [47]:
features_num = resnet18.fc.in_features

In [48]:
features_num

512

In [49]:
#resnet18.fc = nn.Linear(features_num, BATCH_SIZE)
#resnet18.fc = nn.Linear(BATCH_SIZE, 2048)
resnet18.fc = nn.Linear(2048, BATCH_SIZE)
#resnet18.fc = nn.Sigmoid()

In [50]:
resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [51]:
learning_rate = 0.001

In [52]:
resnet18.state_dict().keys()

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.num_batches_tracked', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', '

In [53]:
#resnet18.state_dict()['conv1.weight'].size()
#resnet18.state_dict()['fc.weight'].size()

In [54]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=learning_rate, momentum=0.9)

In [55]:
#criterion = nn.BCELoss()
#optimizer = optim.Adam(resnet18.parameters(), lr=0.001, betas=(0.9, 0.99))

In [56]:
train_rezult_metrics = train_model(
    num_of_epoch,
    #hwi_conv_neural_net,
    resnet18,
    dataset_loaders,
    dataset_sizes,
    criterion,
    optimizer
)

outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[-1.1564,  1.4683, -0.0508,  ..., -0.5598,  0.4142, -0.2349],
        [-0.4405,  1.6873, -0.4400,  ...,  1.1617,  0.5179, -0.0927],
        [-0.4804,  1.5766,  0.0003,  ..., -0.2836,  0.2267, -0.6935],
        ...,
        [-0.6244,  1.8294,  0.0698,  ...,  0.0516,  1.3869, -0.9173],
        [-0.8394,  1.4064,  0.2718,  ..., -0.1556,  0.7927, -0.0212],
        [-0.2472,  1.0591,  0.4759,  ...,  0.2678,  1.3483, -0.4107]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
labels:
 tensor([1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 1], device='cuda:0')
outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[-0.0851,  2.2225, -0.3048,  ...,  0.2731,  0.8158, -0.4154],
        [ 0.3490,  2.2320, -0.0707,  ...,  0.5271,  0.0156, -0.7348],
        [-0.5090,  1.8367, -0.5432,  ...,  0.3901,  0.3597, -0.5121],
    

outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[14.0798, 10.9090, -0.6377,  ..., -0.4870, -0.5246, -1.1876],
        [14.9636,  9.9438, -0.8529,  ..., -0.7357, -1.1077, -0.9125],
        [14.5839, 12.0491, -0.6081,  ..., -0.9651, -0.4643, -1.4966],
        ...,
        [17.3194, 12.8505, -0.0995,  ..., -1.8849, -0.7478, -0.6129],
        [18.0356, 14.7924, -0.6414,  ..., -1.1697, -1.5040, -1.0498],
        [17.7067, 13.5785, -0.8951,  ..., -1.4206, -1.5703,  0.0049]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
labels:
 tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
        1, 1, 0, 1, 0, 0, 1, 0], device='cuda:0')
outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[15.3092, 13.2876, -1.4551,  ..., -1.0399, -1.0691, -1.1509],
        [16.7280, 15.0217, -1.8404,  ..., -0.5464, -1.5040, -1.2782],
        [15.7687, 13.0706, -0.8387,  ..., -0.6143, -0.8112, -0.9345],
    

outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[16.5870, 16.6344, -0.9414,  ..., -1.5165, -1.4222, -0.9961],
        [18.1071, 18.0323, -0.8181,  ..., -1.1799, -1.5074, -1.1270],
        [17.4677, 17.0855, -1.1678,  ..., -0.5937, -1.5314, -1.0409],
        ...,
        [18.6243, 19.7247, -1.1116,  ..., -1.2156, -1.2416, -1.4104],
        [16.8232, 19.2950,  0.0641,  ..., -1.5939, -1.5762, -1.3613],
        [22.7155, 26.3619, -1.4203,  ..., -2.1611, -2.2640, -0.8941]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
labels:
 tensor([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0], device='cuda:0')
outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[19.6302, 20.1233, -1.5281,  ..., -0.8541, -1.5655, -1.0776],
        [18.3122, 18.0833, -0.6185,  ..., -1.7327, -1.3673, -0.6561],
        [19.9782, 21.2718, -0.7208,  ..., -1.5956, -1.6760, -1.5585],
    

outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[20.4260, 18.1968, -1.4231,  ..., -0.6563, -1.1449, -0.5820],
        [21.5049, 19.6688, -1.6319,  ..., -1.5620, -1.2101, -0.8787],
        [19.4872, 16.0791, -0.9439,  ..., -1.5453, -2.1141, -0.5755],
        ...,
        [21.2898, 19.1762, -0.9263,  ..., -0.6964, -1.5843, -1.2633],
        [26.3913, 21.9378, -1.4570,  ..., -1.6475, -2.3133, -1.0618],
        [20.2381, 17.4355, -1.3884,  ..., -0.2530, -1.3702, -1.5121]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
labels:
 tensor([0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 1], device='cuda:0')
outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[18.5049, 18.4657, -1.5037,  ..., -0.8437, -1.2860, -0.7826],
        [22.3338, 19.5264, -1.1659,  ..., -1.5475, -1.4443, -1.0621],
        [19.7935, 18.6937, -0.5676,  ..., -1.3530, -1.5268, -0.5245],
    

outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[21.4112, 19.2714, -1.6597,  ..., -0.2436, -1.4367, -1.5492],
        [21.1371, 23.1258, -0.8224,  ..., -1.4803, -2.0432, -0.6191],
        [20.2824, 20.7484, -1.1100,  ..., -1.3689, -0.9094, -0.4053],
        ...,
        [20.6201, 21.2040, -1.0940,  ..., -0.8779, -1.5397, -0.9609],
        [22.3939, 22.6899, -1.4251,  ..., -0.9953, -1.5952, -0.9599],
        [20.1121, 23.6693, -0.8458,  ..., -1.9063, -2.1907, -1.4571]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
labels:
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 1], device='cuda:0')
outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[18.1536, 17.7165, -1.4790,  ..., -0.8314, -0.8170, -1.1629],
        [19.6012, 20.2124, -1.1163,  ..., -1.3696, -2.4257, -1.2689],
        [22.6522, 20.1286, -0.9310,  ..., -2.5222, -1.8953, -0.9170],
    

...

outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[ 7.6246,  8.0347, -0.5888,  ..., -0.3461, -0.9566, -0.7104],
        [11.5734,  9.8385, -0.7630,  ..., -0.7855, -0.9622, -0.7725],
        [13.9150, 11.7094, -1.0919,  ..., -0.5324, -1.1391, -1.2009],
        ...,
        [ 9.0085,  7.9967, -0.4322,  ..., -0.5501, -0.4606, -0.6400],
        [11.5880, 11.5877, -0.7659,  ..., -0.8769, -1.1957, -0.9038],
        [11.6677,  9.7919, -0.4326,  ..., -1.0100, -1.0053, -0.6066]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
labels:
 tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 0], device='cuda:0')
outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[ 7.0469,  7.9088, -0.6061,  ..., -0.2923, -0.7880, -0.4249],
        [12.4921,  9.5839, -0.4387,  ..., -0.9380, -1.0073, -0.9483],
        [11.0707, 10.0931, -0.5608,  ..., -0.7781, -0.7417, -0.8636],
    

outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[12.1079, 11.3062, -0.5828,  ..., -0.9855, -0.7732, -0.5279],
        [12.1367, 11.7826, -1.1618,  ..., -0.7745, -1.2667, -0.9039],
        [13.8115, 12.3532, -1.0711,  ..., -0.6288, -1.2596, -0.5479],
        ...,
        [10.8329,  9.4610, -0.4575,  ..., -0.6658, -0.7772, -0.6592],
        [10.9051, 10.4653, -1.0054,  ..., -0.5708, -1.2606, -0.5060],
        [12.7675, 10.7277, -0.6946,  ..., -0.7633, -0.8333, -1.0412]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
labels:
 tensor([0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0], device='cuda:0')
outputs.size():
 torch.Size([32, 32])
labels.size():
 torch.Size([32])
outputs:
 tensor([[10.8512,  9.6718, -0.8607,  ..., -0.6804, -0.8269, -0.6416],
        [12.2536,  8.6479, -0.7238,  ..., -0.7779, -0.7347, -0.5247],
        [14.1338, 10.2559, -0.7604,  ..., -0.6521, -1.4338, -0.2973],
    

Process Process-12:
Traceback (most recent call last):
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/uldo/work/kaggle/competitions/Humpback_Whale_Identification/code/load_data.py", line 242, in __getitem__
    image = Image.open(path_to_image_file).convert('RGB')
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/PIL/Image.py", line 892, in convert
    self.lo

Traceback (most recent call last):
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-56-f4184539a75a>", line 8, in <module>
    optimizer
  File "<ipython-input-19-6e493d28ff43>", line 20, in train_model
    train_loss, train_accuracy = one_epoch_train(model, train_loader, criterion, optimizer)
  File "<ipython-input-15-f9c54d3eeca6>", line 7, in one_epoch_train
    for i, data_batch in enumerate(data_loader, 0):
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 330, in __next__
    idx, batch = self._get_batch()
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 309, in _get_batch
    return self.data_queue.get()
  File "/home/uldo/miniconda3/envs/DS-New/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res

KeyboardInterrupt: 

In [ ]:
train_losses, validation_losses, train_accuracies, validation_accuracies = train_rezult_metrics

In [ ]:
plt.plot(np.arange(len(train_losses)), train_losses, label='train')
plt.plot(np.arange(len(validation_losses)), validation_losses, label='validation')
plt.legend()
plt.title("loss by epoch")
plt.show()

plt.plot(np.arange(len(train_accuracies)), train_accuracies, label='train')
plt.plot(np.arange(len(validation_accuracies)), validation_accuracies, label='validation')
plt.legend()
plt.title("accuracy by epoch")
plt.show()

In [ ]:
train_rezult_metrics